In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from dotenv import load_dotenv

# Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# Data Encoding and Scaling
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin

# Pipeline
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import set_config

# Clustering
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from functools import lru_cache
import pickle

In [2]:
warnings.filterwarnings('ignore')

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [4]:
# Loading .env file into my python code
load_dotenv()

True

In [5]:
def create_connection():
    print('creating connection with DB')
    user = os.getenv("DB_USER")
    raw_password = os.getenv("DB_PASSWORD")
    password = quote_plus(raw_password)
    host = os.getenv("DB_HOST")
    port = os.getenv("DB_PORT")
    db = os.getenv("DB_NAME")

    # Credentials of mySQL connection
    connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}"
    engine = create_engine(connection_string)
    print('connection created successfully')
    return engine

In [6]:
def create_df(engine,catcher_only=False):
    print('creating DFs=============')
    if catcher_only:
        print('Catcher DF is requested')
        catcher_df = pd.read_sql('Select jobma_catcher_id, is_premium, jobma_catcher_parent, jobma_verified, subscription_status, company_size FROM jobma_catcher', con=engine) 
        return catcher_df
    print("Wallet DF")
    wallet_df = pd.read_sql('Select catcher_id, is_unlimited FROM wallet', con=engine)
    print("Subscription DF")
    subscription_df = pd.read_sql('Select catcher_id, currency, subscription_amount FROM subscription_history', con=engine)
    print("Invitation DF")
    invitation_df = pd.read_sql('Select jobma_catcher_id, jobma_interview_mode, jobma_interview_status FROM jobma_pitcher_invitations', con=engine)
    print("Job posting DF")
    job_posting_df = pd.read_sql('Select jobma_catcher_id FROM jobma_employer_job_posting', con=engine)
    print("kit DF")
    kit_df = pd.read_sql('Select catcher_id FROM job_assessment_kit', con=engine)
    print('Login DF')
    login_df = pd.read_sql('Select jobma_role_id, jobma_user_id, jobma_last_login FROM jobma_login',con=engine)
    # Closing the Connection
    engine.dispose()
    return wallet_df,subscription_df,invitation_df,job_posting_df,kit_df,login_df

# Specific Methods

**To Fetch Columns from different tables and fitting those functions into Pipeline**

In [8]:
# catcher_df

def fetching_catcher_df(catcher_df):
    print("Processing catcher DF")
    catcher_df = catcher_df[['jobma_catcher_id', 'is_premium', 'jobma_catcher_parent', 'jobma_verified', 'subscription_status', 'company_size']]
    catcher_df['jobma_verified'] = catcher_df['jobma_verified'].replace({'0':0, '1':1})
    catcher_df.drop(catcher_df[catcher_df['is_premium'] == ''].index, inplace=True)
    catcher_df['is_premium'] = catcher_df['is_premium'].replace({'0':0, '1':1})
    catcher_df['subscription_status'] = catcher_df['subscription_status'].replace({'0':0, '1':1, '2':0})

    return catcher_df

In [9]:
# wallet_df

def fetching_wallet_df(wallet_df):
    print("Processing wallet DF")
    wallet_df.rename(columns={'catcher_id': 'jobma_catcher_id'}, inplace=True)
    wallet_df = wallet_df[['jobma_catcher_id', 'is_unlimited']]
    wallet_df['is_unlimited'] = wallet_df['is_unlimited'].replace({'0':0, '1':1})
    wallet_df.drop(wallet_df[wallet_df['is_unlimited'] == ''].index, inplace=True)
    wallet_df.drop_duplicates(inplace=True)

    return wallet_df

In [10]:
# subscription_df

def fetching_subscription_df(subscription_df):
    print("Processing subscription DF")
    subscription_df.rename(columns={'catcher_id': 'jobma_catcher_id'}, inplace=True)
    subscription_df.loc[subscription_df['currency'] == '1', 'subscription_amount'] /= 85.23
    subscription_df = subscription_df.groupby('jobma_catcher_id').agg(
        subscription_amount_in_dollars = ('subscription_amount', 'sum'),
        number_of_subscriptions = ('subscription_amount', 'count'),
    ).reset_index()
    subscription_df.drop_duplicates(inplace=True)

    return subscription_df

In [11]:
# login_df

def fetching_login_df(login_df):
    print("Processing login DF")
    login_df = login_df[login_df['jobma_role_id'] == 3].copy()
    login_df.rename(columns={'jobma_user_id': 'jobma_catcher_id'}, inplace=True)

    # Calculating Number of Gaps between last login and today
    login_df['jobma_last_login'] = pd.to_datetime(login_df['jobma_last_login'], errors='coerce')
    login_df['days_since_last_login'] = (pd.Timestamp('today') - login_df['jobma_last_login']).dt.days
    login_df['days_since_last_login'].fillna(5, inplace=True)
    login_df['days_since_last_login'] = login_df['days_since_last_login'].astype(int)

    # Binning
    bins = [-1,7,30,90,180,365,float('inf')]
    labels = ['Less than 1 Week', '1-4 Weeks', '1-3 Months', '3-6 Months', '6-12 Months', 'More than 1 Year']
    login_df['days_since_last_login'] = pd.cut(login_df['days_since_last_login'], bins=bins, labels=labels)
    login_df = login_df[['jobma_catcher_id', 'days_since_last_login']]

    return login_df

In [12]:
def fetching_features(invitation_df, job_posting_df, kit_df):
    print("Fetching features")
    for df in [invitation_df, job_posting_df, kit_df]:
        if 'catcher_id' in df.columns:
            df.rename(columns={'catcher_id': 'jobma_catcher_id'}, inplace=True)

    job_posting_df['job_posted'] = job_posting_df['jobma_catcher_id'].map(job_posting_df['jobma_catcher_id'].value_counts())
    kit_df['number_of_kits'] = kit_df['jobma_catcher_id'].map(kit_df['jobma_catcher_id'].value_counts())
    invitation_df['number_of_invitations'] = invitation_df['jobma_catcher_id'].map(invitation_df['jobma_catcher_id'].value_counts())
    invitation_df = invitation_df[invitation_df['jobma_interview_mode'].isin(['1', '2'])].copy()
    interview_counts = invitation_df.groupby(['jobma_catcher_id', 'jobma_interview_mode']).size().unstack(fill_value=0)
    interview_counts = interview_counts.rename(columns={'1': 'number_of_recorded_interviews', '2': 'number_of_live_interviews'})
    invitation_df = invitation_df.merge(interview_counts, on='jobma_catcher_id', how='left')
    invitation_df.drop(invitation_df[invitation_df['jobma_interview_status'] == ''].index, inplace=True)
    invitation_df.drop(invitation_df[invitation_df['jobma_interview_status'] == '0'].index, inplace=True)
    invitation_df['jobma_interview_status'] = invitation_df['jobma_interview_status'].astype(int)
    invitation_df = invitation_df.drop_duplicates()

    # invitation_df = invitation_df[['jobma_catcher_id', 'number_of_invitations']].drop_duplicates()
    job_posting_df = job_posting_df[['jobma_catcher_id', 'job_posted']].drop_duplicates()
    kit_df = kit_df[['jobma_catcher_id', 'number_of_kits']].drop_duplicates()

    return invitation_df, job_posting_df, kit_df

In [13]:
def merging_df(catcher_df, wallet_df, subscription_df, invitation_df, job_posting_df, kit_df, recorded_interview_df, live_interview_df, login_df):
    print("Merging DFs")
    final_df = catcher_df.copy()

    # Left join each table one by one
    final_df = final_df.merge(wallet_df, on='jobma_catcher_id', how='left')
    final_df = final_df.merge(subscription_df, on='jobma_catcher_id', how='left')
    final_df = final_df.merge(invitation_df, on='jobma_catcher_id', how='left')
    final_df = final_df.merge(job_posting_df, on='jobma_catcher_id', how='left')
    final_df = final_df.merge(kit_df, on='jobma_catcher_id', how='left')
    final_df = final_df.merge(recorded_interview_df, on='jobma_catcher_id', how='left')
    final_df = final_df.merge(live_interview_df, on='jobma_catcher_id', how='left')
    final_df = final_df.merge(login_df, on='jobma_catcher_id', how='left')
    final_df.drop_duplicates(inplace=True)

    # For Total Sub
    sub_counts = final_df[final_df['jobma_catcher_parent'] != 0].groupby('jobma_catcher_parent').size()
    final_df['total_sub'] = final_df['jobma_catcher_id'].map(sub_counts).fillna(0).astype(int)

    # For Kits
    sub_kits_sum = final_df[final_df['jobma_catcher_parent'] != 0].groupby('jobma_catcher_parent')['number_of_kits'].sum()
    # final_df.loc[final_df['jobma_catcher_id'].isin(sub_kits_sum.index), 'number_of_kits'] += final_df['jobma_catcher_id'].map(sub_kits_sum)
    kits_mapped = final_df['jobma_catcher_id'].map(sub_kits_sum).fillna(0)
    final_df['number_of_kits'] = final_df['number_of_kits'].fillna(0) + kits_mapped
    final_df['number_of_kits'] = final_df['number_of_kits'].astype(int)

    
    # For Invitations
    sub_invitations_sum = final_df[final_df['jobma_catcher_parent'] != 0].groupby('jobma_catcher_parent')['number_of_invitations'].sum()
    # final_df.loc[final_df['jobma_catcher_id'].isin(sub_kits_sum.index), 'number_of_invitations'] += final_df['jobma_catcher_id'].map(sub_invitations_sum)
    invitations_mapped = final_df['jobma_catcher_id'].map(sub_invitations_sum).fillna(0)
    final_df['number_of_invitations'] = final_df['number_of_invitations'].fillna(0) + invitations_mapped
    final_df['number_of_invitations'] = final_df['number_of_invitations'].astype(int)
    
    # For Job Posted
    sub_job_posted_sum = final_df[final_df['jobma_catcher_parent'] != 0].groupby('jobma_catcher_parent')['job_posted'].sum()
    # final_df.loc[final_df['jobma_catcher_id'].isin(sub_job_posted_sum.index), 'job_posted'] += final_df['jobma_catcher_id'].map(sub_job_posted_sum)
    job_posted_mapped = final_df['jobma_catcher_id'].map(sub_job_posted_sum).fillna(0)
    final_df['job_posted'] = final_df['job_posted'].fillna(0) + job_posted_mapped
    final_df['job_posted'] = final_df['job_posted'].astype(int)

    # For Recorded Interviews
    sub_recorded_sum = final_df[final_df['jobma_catcher_parent'] != 0].groupby('jobma_catcher_parent')['number_of_recorded_interviews'].sum()
    # final_df.loc[final_df['jobma_catcher_id'].isin(sub_recorded_sum.index), 'number_of_recorded_interviews'] += final_df['jobma_catcher_id'].map(sub_recorded_sum)
    recorded_mapped = final_df['jobma_catcher_id'].map(sub_recorded_sum).fillna(0)
    final_df['number_of_recorded_interviews'] = final_df['number_of_recorded_interviews'].fillna(0) + recorded_mapped
    final_df['number_of_recorded_interviews'] = final_df['number_of_recorded_interviews'].astype(int)

    # For Live Interviews
    sub_live_sum = final_df[final_df['jobma_catcher_parent'] != 0].groupby('jobma_catcher_parent')['number_of_live_interviews'].sum()
    # final_df.loc[final_df['jobma_catcher_id'].isin(sub_live_sum.index), 'number_of_live_interviews'] += final_df['jobma_catcher_id'].map(sub_live_sum)
    live_mapped = final_df['jobma_catcher_id'].map(sub_live_sum).fillna(0)
    final_df['number_of_live_interviews'] = final_df['number_of_live_interviews'].fillna(0) + live_mapped
    final_df['number_of_live_interviews'] = final_df['number_of_live_interviews'].astype(int)

    # For Minimum Login Days
    login_order = {
        'Less than 1 Week':0,
        '1-4 Weeks':1,
        '1-3 Months':2,
        '3-6 Months':3,
        '6-12 Months':4,
        'More than 1 Year':5
    }

    final_df['days_since_last_login'] = final_df['days_since_last_login'].map(login_order).fillna(5).astype(int)
    sub_min_login = final_df[final_df['jobma_catcher_parent'] != 0].groupby('jobma_catcher_parent')['days_since_last_login'].min()
    final_df.loc[final_df['jobma_catcher_id'].isin(sub_min_login.index), 'days_since_last_login'] = final_df.loc[final_df['jobma_catcher_id'].isin(sub_min_login.index), 'jobma_catcher_id'].map(sub_min_login)

    verified_df = final_df[final_df['jobma_verified'] == 1].copy()
    df = verified_df[verified_df['jobma_catcher_parent'] == 0].copy()
    df.drop(['jobma_catcher_parent', 'jobma_verified'], axis=1, inplace=True)
    
    compare_df = df.copy()
    df.drop('jobma_catcher_id', axis=1, inplace=True)

    print(f"Final merged df shape is {df.shape}")

    return df, compare_df

In [14]:
# This Function is to fill all missing values

def fill_missing_values(final_df):
    final_df = final_df.copy()
    fill_values = {
        'is_premium': 0,
        'subscription_status': 0,
        'company_size': '1-25',
        'is_unlimited': 0,
        'subscription_amount_in_dollars': 0,
        'number_of_subscriptions': 0,
        'jobma_interview_status': 1,
        'number_of_invitations': 0,
        'number_of_recorded_interviews': 0,
        'number_of_live_interviews': 0,
        'job_posted': 0,
        'number_of_kits': 0,
        'days_since_last_login': 5,
        'total_sub': 0,
    }
    
    return final_df.fillna(fill_values)

In [15]:
# Data Encoding 

def ordinal_encoder(df):
    # ordinal_col = ['company_size', 'days_since_last_login']
    ordinal_col = ['company_size']
    company_size_order = ['1-25', '26-100', '101-500', '500-1000', 'More than 1000']
    # login_days_order = ['Less than 1 Week', '1-4 Weeks', '1-3 Months', '3-6 Months', '6-12 Months', 'More than 1 Year']

    # total_order = [company_size_order, login_days_order]
    ordinal = OrdinalEncoder(categories=[company_size_order])

    encoded = ordinal.fit_transform(df[ordinal_col].astype(str))
    # encoded += 1

    encoded_df = pd.DataFrame(encoded, columns=[f' {col}_ord' for col in ordinal_col], index=df.index)

    df.drop(columns=ordinal_col, inplace=True)

    df = pd.concat([df, encoded_df], axis=1)

    return df

In [16]:
# Log Transformation 

def log_transform(df):
    log_cols = [
        'total_sub',
        'subscription_amount_in_dollars',
        'number_of_subscriptions',
        'number_of_invitations',
        'job_posted',
        'number_of_kits',
        'number_of_recorded_interviews',
        'number_of_live_interviews',
        'days_since_last_login'
    ]

    df = df.copy()
    for col in log_cols:
        if col in df.columns:
            # fill NaNs
            df[col] = df[col].fillna(0)
            # if a number is less than zero, turn it into zero
            df[col] = df[col].clip(lower=0)
            # safe log1p
            df[col] = np.log1p(df[col])

    return df

# Pipeline

In [18]:
class MergingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        engine = create_connection()
        wallet_df,subscription_df,invitation_df,job_posting_df,kit_df,recorded_interview_df,live_interview_df,login_df = create_df(engine)
        self.wallet_df = wallet_df
        self.subscription_df = subscription_df
        self.invitation_df = invitation_df
        self.job_posting_df = job_posting_df
        self.kit_df = kit_df
        self.recorded_interview_df = recorded_interview_df
        self.live_interview_df = live_interview_df
        self.login_df = login_df

    def fit(self, X, y=None):
        return self

    def transform(self, catcher_df):
        catcher = fetching_catcher_df(catcher_df)
        wallet = fetching_wallet_df(self.wallet_df)
        subscription = fetching_subscription_df(self.subscription_df)
        login = fetching_login_df(self.login_df)
        invitation, job_posting, kit, recorded, live = fetching_features(
            self.invitation_df,
            self.job_posting_df,
            self.kit_df,
            self.recorded_interview_df,
            self.live_interview_df
        )
        final_df, compare_df = merging_df(catcher, wallet, subscription, invitation, job_posting, kit, recorded, live, login)
        self.compare_df_ = fill_missing_values(compare_df)
        
        return final_df

In [19]:
class PreprocessingTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = fill_missing_values(X)
        X = ordinal_encoder(X)
        X = log_transform(X)
        return X

In [20]:
# Pipelines

# merge_pipeline = Pipeline([
#     ('merge', MergingTransformer(wallet_df=wallet_df,
#                                  subscription_df=subscription_df,
#                                  login_df=login_df,
#                                  invitation_df=invitation_df,
#                                  job_posting_df=job_posting_df,
#                                  kit_df=kit_df,
#                                  recorded_interview_df=recorded_interview_df,
#                                  live_interview_df=live_interview_df))
# ])

# Step 2: Preprocess merged data
preprocess_pipeline = Pipeline([
    ('preprocessing', PreprocessingTransformer()),
    ('scaler', StandardScaler()),
    ('to_tensor', FunctionTransformer(lambda x: torch.tensor(x, dtype=torch.float32)))
])

# full_pipeline = Pipeline([
#     ('merge_pipeline', merge_pipeline),
#     ('preprocess_pipeline', preprocess_pipeline)
# ])

NameError: name 'wallet_df' is not defined

# Fit Pipeline

In [ ]:
X_tensor = full_pipeline.fit_transform(catcher_df)

In [ ]:
type(X_tensor)

In [ ]:
len(X_tensor)

In [ ]:
X_tensor.dtype

In [ ]:
X_df = pd.DataFrame(X_tensor)

# Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
X_data = CustomDataset(X_tensor)

# DataLoader

In [ ]:
BATCH_SIZE = 16

In [ ]:
dataloader = DataLoader(X_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
len(X_data)

In [ ]:
len(dataloader)

# Define a Model (AutoEncoder in this case)

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self, input_shape):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_shape, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32)  # bottleneck
        )

        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, input_shape)
        )

    def forward(self, X):
        encoded = self.encoder(X)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
# Initializing the Model 

input_shape = X_df.shape[1]
model_1 = AutoEncoder(input_shape)
model_1.to(device)

In [ ]:
# Important Parameters 

learning_rate = 0.001
epochs = 50
patience = 5
delta = 1e-4

best_loss = float('inf')
epochs_no_improve = 0
training_losses = []

In [ ]:
# Loss Function, Optimizers and LR Scheduler 

loss_function = nn.SmoothL1Loss()
optimizer = torch.optim.AdamW(model_1.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

# Training Loop

In [ ]:
for epoch in range(epochs):
    model_1.train()
    epoch_loss = 0

    for batch in dataloader:
        encoded, decoded = model_1(batch)
        loss = loss_function(decoded, batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(dataloader)
    training_losses.append(avg_loss)
    scheduler.step(avg_loss)

    print(f"Epoch: {epoch+1}/{epochs} | Loss: {avg_loss:.6f}")

    # Early Stopping
    if avg_loss < best_loss - delta:
        best_loss = avg_loss
        epochs_no_improve = 0

    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

    # Print current learning rate
    for param_group in optimizer.param_groups:
        print(f"Learning Rate: {param_group['lr']:.6f}")

# Clustering

In [ ]:
model_1.eval()

with torch.no_grad():
    encoder, _ = model_1(X_tensor)

latent_np = encoder.cpu().numpy()

In [ ]:
wcss = []
silhouette_scores = []
k_range = range(2, 11)

In [ ]:
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(latent_np)

    wcss.append(kmeans.inertia_) # Inertia = WCSS
    silhouette_scores.append(silhouette_score(latent_np, labels))

In [ ]:
# Elbow Rule to decide the number of clusters 

torch.manual_seed(42)
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(k_range, wcss, marker='o')
plt.title('Elbow Method (WCSS vs K)')
plt.xlabel('Number of clusters (K)')
plt.ylabel('WCSS (Inertia)')

# Step 5: Plot Silhouette Scores
plt.subplot(1, 2, 2)
plt.plot(k_range, silhouette_scores, marker='o', color='green')
plt.title('Silhouette Score vs K')
plt.xlabel('Number of clusters (K)')
plt.ylabel('Average Silhouette Score')

plt.tight_layout()
plt.show()

In [ ]:
#Initialising KMeans
kmeans = KMeans(n_clusters=4, n_init=50, init='k-means++', random_state=42)
cluster_ids = kmeans.fit_predict(latent_np)
cluster_centers = torch.tensor(kmeans.cluster_centers_, dtype=torch.float32).to(device)

In [ ]:
latent_2d = PCA(n_components=2).fit_transform(latent_np)
plt.figure(figsize=(10, 6))
plt.scatter(latent_2d[:, 0], latent_2d[:, 1], c=cluster_ids, cmap='tab10', s=10)
plt.title("KMeans clusters on latent space (Before DEC)")
plt.colorbar()
plt.show()

## Test it on User's Preferences 

In [ ]:
X_df.columns

In [ ]:
compare_df = merge_pipeline.named_steps['merge'].compare_df_

In [ ]:
compare_df.info()

In [ ]:
compare_df.columns

In [ ]:
user_pref_good = {'is_premium':1,
                 'subscription_status':1,
                 'company_size':'More than 1000', 
                 'is_unlimited':1,
                 'subscription_amount_in_dollars': 10000.00,
                 'number_of_subscriptions':2,
                 'number_of_invitations':25,
                 'job_posted':10,
                 'number_of_kits':7,
                 'number_of_recorded_interviews':18,
                'number_of_live_interviews':15,
                'days_since_last_login':5,
                'total_sub':3,
            }

In [ ]:
def recommend(user_input, model, latent_embeddings, compare_df, pipeline, top_k=5):
    # Transform the user input using the pipeline
    user_df = pd.DataFrame([user_input])
    user_input_tensor = pipeline.transform(user_df)

    model.eval()
    with torch.no_grad():
        # Get the user embedding from the model
        user_embedding, _ = model(user_input_tensor)

        # Normalize the user embedding
        user_embedding = F.normalize(user_embedding, p=2, dim=1)

        # Convert latent embeddings to tensor and normalize
        latent_embeddings_tensor = torch.tensor(latent_embeddings, dtype=torch.float32)
        latent_embeddings_tensor = F.normalize(latent_embeddings_tensor, p=2, dim=1)

        # Calculate cosine similarity
        similarities = F.cosine_similarity(user_embedding, latent_embeddings_tensor, dim=1)

        # Get top K most similar indices
        top_indices = similarities.topk(top_k).indices.cpu().numpy()

        # Get the top K recommendations from the compare_df
        recommended = compare_df.iloc[top_indices].copy()
        recommended['similarity'] = similarities[top_indices].cpu().numpy()

    return recommended

In [ ]:
result = recommend(user_pref_good, model_1, latent_np, compare_df, preprocess_pipeline, top_k=5)
result

In [ ]:
user_pref_test = {'is_premium':1,
                 'subscription_status':1,
                 'company_size':'1-25',
                 'is_unlimited':1,
                 'subscription_amount_in_dollars': 125.0,
                 'number_of_subscriptions':1,
                 'number_of_invitations':18,
                 'job_posted':3,
                 'number_of_kits':3,
                 'number_of_recorded_interviews':3,
                'number_of_live_interviews':1,
                'days_since_last_login':4,
                'total_sub':2
            }

In [ ]:
result1 = recommend(user_pref_test, model_1, latent_np, compare_df, preprocess_pipeline, top_k=10)
result1